# Анализ международного рынка видеоигр

Наш проект состоит в исследовании различных особенностей данных о международных продажах видеоигр.<br>
Нами будет проведена обработка соответствующего набора данных, на основании которой мы исследуем параметры, присущие каждой проданной игре (жанр, оценку критиков и пользователей, игровую платформу и .т.д.), которые потенциально могут влияет на ее продажи.<br>
Кроме этого, мы составим приблизительный портрет пользователя из каждого региона, по которому есть данные о продажах, исследовав популярные в этом регионе игровые платформы, жанры, также будет проведен анализ самых часто встречающихся игровых рейтингов в каждом из этих регионов.<br>
Наконец, будут проведены проверки статистических гипотез о средних пользовательских оценках игр, выпущенных под определенную платформу, имеющих тот или иной жанр.<br>
Результатом проекта станут некоторые предположения о том, какие из параметров объектов из набора данных потенциально способны влиять на его продажи.

## 1. Обзор и подготовка данных

Для начала импортируем необходимые библиотеки:
1. pandas для работы с набором данных
2. numpy для работы с NaN и массивами
3. plotly для интерактивной визуализации
4. missingno для визуального анализа пропусков в данных
5. scipy для проверки гипотез

In [ ]:
#!pip install missingno

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import missingno as msno
from scipy import stats as st

Теперь сохраним датасет в переменную games:

In [ ]:
try:
    games = pd.read_csv('games.csv')
except:
    games = pd.read_csv('/datasets/games.csv')

Посмотрим, какой вид имеет набор данных, для чего выведем его первые 5 строк:

In [ ]:
games.head()

Как видим, данные разбиты на три информационных блока. *Первый* состоит из основной информации об игре - ее название, платформа, для которой она выпущена, года релиза и игрового жанра. *Второй* содержит в себе данные о продажах игр по регионам - Северная Америка, Европейский Союз, Япония, другие продажи. В *третьем* блоке расположены данные об оценках критиков, пользователей и о рейтинге игры. 

**Приведем названия столбцов к нижнему регистру**:

In [ ]:
games.columns = games.columns.str.lower()

Получим общую информацию о датафрейме и **осуществим преобразование типов**:

In [ ]:
games.info()

Заметим, что в столбце 'user_score' встречается строковое значение 'tbd', представляющее собой аббревиатуру от to be decided/to be determined ([источник](https://dictionary.cambridge.org/dictionary/english/tbd))<br>
Проще говоря, значение пользовательствого рейтинга в этих строках неизвестно. Тогда заменим его на NaN:

In [ ]:
games['user_score'].replace('tbd', np.nan, inplace=True)

После приведем этот же столбец к типу float:

In [ ]:
games['user_score'] = games['user_score'].astype('float')

Наконец, проведем нисходящее преобразование числовых данных, чтобы оптимизировать расход памяти:

In [ ]:
games_float = games.select_dtypes(include=['float'])#Пользуемся методом DataFrame.select_dtypes() для выбора столбцов, 
                                                    #хранящих числа с плавающей точкой
converted_float = games_float.apply(pd.to_numeric, downcast='float')#Осуществляем нисходящее преобразование типов
games[converted_float.columns] = converted_float#Перезаписываем соответствующие столбцы в games

Уточним формат отображения вещественных чисел:

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

Теперь **совершим обработку пропусков**, для чего оценим их количество в датафрейме:

In [ ]:
msno.bar(games)

Как видно, все колонки кроме тех, что содержат информацию об игровой платформе и продажах, содержат то или иное количество пропусков, особенно большим оно является в столбцах с оценкой и рейтингом игр. 

Для начала разберемся с пропусками в столбцах 'name' и 'genre' - их всего по 2 штуки в каждом. Выведем строки с пропусками в этих столбцах:

In [ ]:
games.query('name.isna() or genre.isna()')

Как видим, таких строк 2 - в каждой значения нужных нам столбцов пропущены. Кроме того, пропущены и значения в столбцах, содержащих информацию об оценке и рейтинге игр. Предположим, что эти пробелы вызваны ошибкой сбора информации о продажах. Скажем, что эти две строки дают нам слишком мало информации и удалим их:

In [ ]:
games = games.dropna(subset=['name', 'genre'])

Теперь обратимся к столбцу 'year_of_release'. Кажется, что пропуски в нем хотя бы частично можно было заполнить годом выхода той же игры, но на другой платформе. Проблема заключается в том, что зачастую версии игры на разных плафтормах могут выходить с разницей в год и больше. Примем решение оставить пропуски, а не заполнять их неточными данными.

По той же логике, хотя их и можно заполнить медианным значением по имени игры или платформе, не будем трогать столбцы 'critic_score' и 'user_score'. А вот в столбце 'rating' заменим пропуски строкой 'unknown':

In [ ]:
games['rating'] = games['rating'].fillna('unknown')

Причиной пропусков в столбцах 'year_of_release', 'critic_score' и 'user_score' служат, судя по всему, ошибки в сборе данных или в формировании датафрейма. 

Наконец, **создадим столбец с суммарными продажами** во всех регионах:

In [ ]:
games['total_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']

В качестве обработки данных, предваряющих исследование, названия столбцов были приведены к нижнему регистру. Затем провели обзор типов данных, которыми записываются значения каждого столбца, типы для числовых столбцов были заменены в целях оптимизации использования памяти. После были исследованы пропуски в датафрейме, для столбцов 'name', 'genre' они были удалены, для столбца 'rating' - заменыны на 'unknown', для остальных было принято решение о целесообразности "оставить все как есть", чтобы не вносить в датафрейм неточные данные, могущие навредить на этапе анализа. 

## 2. Исследовательский анализ данных

**Посмотрим, сколько игр выпускалось в разные годы**.

Создадим таблицу, где индексами будут годы релиза, а значениями - количество выпущенных игр:

In [ ]:
games_by_year = games.pivot_table(index='year_of_release', values='name', aggfunc='count')

Построим по этой таблице столбчатую диаграмму:

In [ ]:
fig = px.bar(games_by_year, x=games_by_year.index, y='name', title='Выпуск игр по годам')
fig.show()

Для более детального понимания выведем еще и статистические характеристики для этой таблицы (из процентилей выведем только медиану):

In [ ]:
games_by_year.describe(percentiles=[.5])

Как видим, размах данных весьма велик - от 9 игр, выпущенных в 1980 году, до 1427, выпущенных в 2008. В среднем за 37 лет, доступных нам для анализа, за год выходило 338 игр (медианное значение, не зависящее от сильного разброса). Активный рост выпуска игр начался с 1990 года и завершился в 2008, где количество игровых релизов, как мы уже определили, достигло максимума. Затем последовало снижение выпуска вплоть до 2013 года, с которого наблюдаем рост, происходящий низким темпом до 2015 года, после чего снова видим падение выпуска (но не будем забывать, что данные за 2016 могут быть неполными). Похоже что для прогнозов на будущие года, нам не понадобятся "слишком старые" данные. Уточним данную интуицию далее.

**Проведем анализ изменения продаж по платформам**. Сначала построим таблицу, которая поможет определить лидеров по продажам среди разных платформ:

In [ ]:
sales_by_platform = games.pivot_table(index='platform', values='total_sales', aggfunc='sum').sort_values(by='total_sales', ascending=False)
sales_by_platform

Возьмем для анализа первую десятку по продажам и построим для них сводную таблицу:

In [ ]:
top_platforms = ['PS2', 'X360', 'PS3', 'Wii', 'DS', 'PS', 'GBA', 'PS4', 'PSP', 'PC']#Создаем список из первых 10 платформ
games_for_top_platforms = games.query('platform in @top_platforms')#По нему создаем срез оригинального набора

sales_by_top_platforms = games_for_top_platforms.pivot_table(columns='platform', index='year_of_release',#Строим сводную
                                                             values='total_sales', aggfunc='sum')#таблицу

Теперь же для этой таблицы выведем столбчатую диаграмму с разделением каждого столбца по платформам:

In [ ]:
fig = px.bar(sales_by_top_platforms, x=sales_by_top_platforms.index, 
             y=['DS', 'GBA', 'PC', 'PS', 'PS2', 'PS3', 'PS4', 'PSP', 'Wii', 'X360'],
             title='Выпуск игр по годам для 10 самых популярных платформ')
fig.update_xaxes(range=[1991,2017])
fig.show()

На графике видим следующую "живучесть" платформ: 
PC - 24 года, PS - 9 лет, PS2 - 11 лет, GBA - 6 лет, DS - 9 лет, PSP - 10 лет, X360 - 11 лет, PS3 - 11 лет, Wii - 9 лет и PS4 - 4 года. Медианное значение этого набора - 9.5 лет. То есть, игровая платформа остается актуальной 9-10 лет.

In [ ]:
years = pd.Series([24, 9, 11, 6, 9, 10, 11, 11, 9, 4])#Создаем серию из продолжительности "жизни" популярных платформ
years.median()#Находим медианную "живучесть"

Еще раз взглянув на графики, заметим, что период активного роста продаж для каждой платформы составляет примерно половину ее "жизненного цикла" - 3-4 года. Приняв это во внимание, *выберем в качестве актуального периода для исследования четырехлетний интервал с 2013 по 2016 год* - в эти годы сохраняются тендеции для основных платформ, активных на момент 2011 года (тенденции к падению продаж), и начинают проявляться новые (появление и рост продаж для PS4). 

Оставим в нашем наборе лишь данные за необходимые года и **проведем обзор и сравнение продаж по каждой платформе за этот период**.

Чтобы получить данные за актуальный период по всем платформам:

In [ ]:
sales_by_top_platforms_actual = (games
                                 .query('year_of_release >= 2013 and year_of_release <= 2016')
                                 .pivot_table(index='year_of_release', columns='platform', values='total_sales', aggfunc='sum'))

Взглянем на получившийся датафрейм:

In [ ]:
sales_by_top_platforms_actual

Заметим, что по платформе DS  в этот пеирод нам доступно лишь одно наблюдение, так что этот столбец удаляем:

In [ ]:
sales_by_top_platforms_actual = sales_by_top_platforms_actual.drop('DS', axis=1)

In [ ]:
sales_by_top_platforms_actual.columns

Теперь построим линейный график, где каждому году нужного нам периода будет соответствовать суммарная годовая продажа игр для каждой платформы, если игры для нее выходили в этом году: 

In [ ]:
fig = px.line(sales_by_top_platforms_actual, x=sales_by_top_platforms_actual.index, 
              y=['3DS', 'PC', 'PS3', 'PS4', 'PSP', 'PSV', 'Wii', 'WiiU', 'X360', 'XOne'], markers=True,
              title='Продажи игр за 2013-2016 годы')
fig.show()

Как следует из графика, в начале наблюдений лидерами по продажам были платформы PS3, X360 и 3DS, в 2014 году уступившие PS4 и XOne, которые стремительно выросли в продажах за 2013 год. Информации на 2016 год нет об играх для PSP, что, как мы помним, может быть вызвано неполнотой данных за этот год. С другой стороны, даже если предположить, что продажи игр на PSP в 2016 году были - скорее всего, их сумма была весьма низкой, ведь эта платформа демонстрировала постоянное падение продаж, которые в 2015 году оказались в нуле.<br>
Видим, что, за некоторыми временными исключениями (рост продаж игр под PS4 и XOne в 2013-2014 годах), график демонстрирует устойчивую тенденцию к падению выручки от игр для всех платформ. Ниже всего она у игр на таких платформах как PSP, Wii, PSV и X360, которая со второго места по продажам в 2013 к концу наблюдаемого периода спустилась на предпоследнее. <br>
*Среди тех, что в 2015-2016 годах показывали наибольшие среди остальных продажи, и могут потенциально остаться среди лидеров и дальше, - игры для платформ PS4, XOne, и 3DS*. PS4 являетя абсолютным лидером почти весь период наблюдений, XOne - уверенно располагается на втором месте, 3DS благодаря относительно плавному падению продаж оказалась в 2015-2016 годах на том же третьем месте, на котором мы ее застали в 2013. Остальные же платформы, как видно, постепенно теряли в продажах, к 2016 году образовав кластер у самой оси абсцисс с продажами от 0.18 до 5.25. 

Теперь **построим диаграмму размаха по суммарным продажам для потенциально прибыльных платформ**.

Создадим список перспективных, актуальных платформ:

In [ ]:
actual_platfroms = ['PS4', 'XOne', '3DS']

По этому списку создадим новый датафрейм - срез оригинального, в который попадут лишь игры для перспективных платформ, выпущенные с 2013 до 2016 года:

In [ ]:
games_actual = games.query('platform in @actual_platfroms and year_of_release >= 2013 and year_of_release <= 2016')
games_actual = games_actual.reset_index()#Обновим индекс получившегся набора данных

Создадим таблицу, где в качестве столбцов будут использоваться платформы, а в качестве значений - суммарная выручка конкретной игры (индекс оставим как в датафрейме games):

In [ ]:
sales_by_actual_platforms = games_actual.pivot_table(index='index', columns='platform', values='total_sales')

Затем построим "ящики с усами", для удобства анализа отобразив на них сами данные в виде точек, и ограничив их по оси ординат:

In [ ]:
fig = px.box(sales_by_actual_platforms, y=['PS4', 'XOne', '3DS'], 
              points="all", title='Диаграммы размаха продаж игр для перспективных платформ')
fig.update_yaxes(range=[0,1.8])
fig.show()

На диаграмме видим, что медианные продажи выше всего у игр для XOne, почти не отличаются от них медианные продажи для PS4, хуже продаются игры для 3DS. Самый широкий интерквартильный размах у графика продаж игр на PS4, самый узкий - снова у 3DS. Заметим и скошенность данных вправо для всех платформ. В каждом графике видим большое количество выбросов (чтобы их увидеть, можно ослабить ограничение по оси y в 3 строке кода) - необычно высоких продаж той или иной игры.<br>
Скажем, что из имеющихся данных (помним, что они неполны), можно сделать вывод о (в среднем) несколько больших продажах игр, выпущенных на платформе XOne. 

**Посмотрим, как влияют на продажи игр для платформы PS4 отзывы пользователей и критиков.** Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами. Для аппроксимации точечных значений нашего графика воспользуемся обычным методом наименьших квадратов ([ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares)).

Для построения диаграммы сперва создадим срез датафрейма games_actual, где ограничим представленные платформы той, по которой будем строить график, - PS4:

In [ ]:
score_impact_PS4 = games_actual.query('platform == "PS4"')

Теперь по этому срезу построим график, показывающий связь между оценкой критиков и суммарной выручкой:

In [ ]:
fig = px.scatter(score_impact_PS4, x="critic_score", y="total_sales", 
                 trendline="ols", title='Влияние оценок критиков на продажи игр для PS4')
fig.show()

Кроме того, выведем коэффициент корреляции Пирсона между этими величинами:

In [ ]:
score_impact_PS4['total_sales'].corr(score_impact_PS4['critic_score'])

Наблюдается небольшая положительная корреляция. Проверим то же для пользовательской оценки. Построим график по тому же срезу:

In [ ]:
fig = px.scatter(score_impact_PS4, x="user_score", y="total_sales", 
                 trendline="ols", title='Влияние оценок пользователей на продажи игр для PS4')
fig.show()

Выведем соответствующий коэффициент корреляции:

In [ ]:
score_impact_PS4['total_sales'].corr(score_impact_PS4['user_score'])

С уверенностью заключим, что влияние оценок критиков на продажи игр под PS4 сильнее, чем влияние пользовательских оценок - корреляция с последними почти нулевая.

**Соотнесем выводы с продажами игр на других платформах.**

Создадим другой срез, в него попадут все платформы, кроме PS4, которую мы уже рассмотрели:

In [ ]:
score_impact_other = games_actual.query('platform != "PS4"')

Для выявления связи оценки критиков и суммарной выручки строим фасеточную диаграмму рассеяния - в аргумент facet_col передадим сразу все три значения столбца 'platform' нашего среза:

In [ ]:
fig = px.scatter(score_impact_other, x="critic_score", y="total_sales", facet_col="platform", 
                 trendline="ols", title='Влияние оценок критиков на продажи игр')
fig.show()

С помощью цикла узнаем соответствующие коэффициенты корреляции:

In [ ]:
other_platforms=['3DS', 'XOne']
for platform in other_platforms:
    games_temp = games_actual.query('platform == @platform')
    print(f'Корреляция для платформы {platform} -', games_temp['total_sales'].corr(games_temp['critic_score']))

Заметим, что коэффициенты для этих платформ сравнимы с коэффициентом для PS4, у 3DS он ниже, у XOne - несколько выше.

Теперь построим такой же график уже для визуализации связи общей выручки и оценки пользователей:

In [ ]:
fig = px.scatter(score_impact_other, x="user_score", y="total_sales", facet_col="platform", 
                 trendline="ols", title='Влияние оценок пользователей на продажи игр')
fig.show()

Используя тот же цикл, выведем значения коэффициента корреляции Пирсона:

In [ ]:
for platform in other_platforms:
    games_temp = games_actual.query('platform == @platform')
    print(f'Корреляция для платформы {platform} -', games_temp['total_sales'].corr(games_temp['user_score']))

Для Xone, так же как в случае с PS4, зависимость продаж от пользовательских оценок находится в нуле. Для платформы 3DS же наблюдаем небольшую положительную корреляцию.

В результате анализа скажем, что зависимость продаж от оценок пользователей прослеживается (положительная, хоть и весьма слабая) только для платформы 3DS, тогда как зависимость от оценок критиков прослеживается для всех платформ в немного разной степени - происходит падение коэффициента корреляции Пирсона в порядке XOne, PS4, 3DS.

**Рассмотрим общее распределение прибыли по жанрам игр.**

Сначала узнаем, как обстоит дело с общей (суммарной) выручкой по каждому из жанров игр. 

Построим график по специально созданной таблице с жанрами в качестве индексов и отсортированной в порядке убывания суммарной выручкой в качестве значений:

In [ ]:
sales_by_genre = (games_actual
                  .pivot_table(index='genre', values='total_sales', aggfunc='sum')
                  .sort_values(by='total_sales', ascending=False))

In [ ]:
fig = px.bar(sales_by_genre, x=sales_by_genre.index, 
             y='total_sales',
             title='Выручка, полученная играми каждого жанра')
fig.show()

Итог этого шага таков: самые продающиеся жанры - action, shooter и role-playing, а жанры с наименьшими продажами - simulation, strategy и puzzle (написаны в порядке убывания суммарной выручки). 

Теперь же вравним медианные продажи по жанрам с помощью диаграммы размаха, чтобы получить более точный результат.

Создадим альтернативную таблицу, где жанры будут расположены в названиях столбцов и построим по всем столбцам диаграмму размаха:

In [ ]:
sales_by_genre_alternative = games_actual.pivot_table(index='index', columns='genre', values='total_sales')

In [ ]:
fig = px.box(sales_by_genre_alternative, y=['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
             'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports','Strategy'], 
              points="all", title='Диаграммы размаха для продаж игр по жанрам')
fig.update_yaxes(range=[0,5.7])
fig.show()

Для того, чтобы лучше рассмотреть совсем маленькие ящики, можно воспользоваться инструментом графика 'Zoom'.

После чего выведем специально созданную таблицу, в которой каждому жанру в индексе соответствует медиана продаж:

In [ ]:
sales_by_genre_alternative_median = games_actual.pivot_table(index='genre', values='total_sales', aggfunc='median')
sales_by_genre_alternative_median.sort_values(by='total_sales', ascending=False)

Как видим, распределение после уточнения изменилось: в первой тройке по продажам теперь находятся жанры shooter, sports и platform, а в последней - puzzle, strategy и adventure. Как видим, первая тройка отличается от предыдущего результата весьма ощутимо, последняя тройка же изменила свой порядок, кроме того, вместо жанра simulation в ней оказался жанр adventure. Так мы избежали ошибочного результата, проведя более аккуратное сравнение по медианам. 

## 3. Создание портрета потребителя для каждого региона

Для начала создадим функции, первая по названию параметра возвращает таблицу с рейтингом этого параметра в каждом из регионов, вторая - рисует 3 столбчатых графика.

Первой напишем функцию top_5, которая составит таблицу на 3 пары столбцов вида "параметр" - "его доля в купленных играх". Заметьте особые условия для параметра 'rating' - для него не добавляется строка 'other', поэтому в финальном датафрейме изначально 5 строк, а не 6. Перед написанием функции сделаем срез оригинального датафрейма, в котором будем учитывать лишь игры, выпущенные в актуальный период.

In [ ]:
games_actual_period = games.query('year_of_release >= 2013 and year_of_release <= 2016')

In [ ]:
def top_5(param):
    if param == 'rating':
        
        top_param = pd.DataFrame({'to_be_deleted': [np.nan] * 5})#Финальный датафрейм, куда будем добавлять столбцы

    else:
        
        top_param = pd.DataFrame({'to_be_deleted': [np.nan] * 6})#Финальный датафрейм, куда будем добавлять столбцы
        
    regional_param = games_actual_period.groupby(param)[['na_sales', 'eu_sales', 'jp_sales']].sum()
                                                                  #Группируем данные по параметру и
                                                                  #суммируем прибыль в каждом регионе

    for regional_sales in ['na_sales', 'eu_sales', 'jp_sales']:#Запускаем цикл по каждому региону
        
        regional_param[regional_sales] /= regional_param[regional_sales].sum()#Находим долю игр, купленных 
                                                                                      #в связи с этим параметром

        top_param_current = pd.Series(regional_param[regional_sales]#Создающим текущий топ-5 параметров для региона
                                          .sort_values(ascending=False)
                                          .head())

        if param != 'rating':
            top_param_current['other'] = (regional_param[regional_sales]#Добавляем строку с суммарной долей
                                              .sort_values(ascending=False)#остальных платформ
                                              .iloc[5:]
                                              .sum())

        top_param_current = top_param_current.reset_index()#Обновляем индекс
        
        top_param = top_param.merge(top_param_current, left_index=True, right_index=True)#Добавляем получившиеся
                                                                                            #столбцы в итоговый датафрейм

    top_param = top_param.dropna(axis=1, how='all')#Удаляем из датафрейма пустой столбец, использованный для 
                                                           #более удобного соединения таблиц
    top_param.columns = (['na_top', 'na_sales_proportion',#Переименовываем столбцы в соответствии 
                          'eu_top', 'eu_sales_proportion',#с нужными регионами
                          'jp_top', 'jp_sales_proportion'])
    return top_param

Теперь напишем простую функцию bar_plot, которая будет рисовать 3 столбчатые диаграммы, где на оси абсцисс будет располагаться название параметра, а на оси ординат - его доля в суммарных покупках.<br>
Автор использует для визуализации библиотеку plotly express, которая рисует 3 отдельных графика, а не другие библиотеки, которые могут рисовать графики на тех же осях, потому что даже при этом plotly обладает большим преимуществом - интерактивностью и панелью инструментов, с помощью которых можно проследить нужное значение на графике, приблизить или отдалить какой-либо регион графика и т.д. (Кроме того стоит отметить и важность такой характеристики как "привычка" в этом вопросе).

In [ ]:
def bar_plot (parameter):
    for region in ['na', 'eu', 'jp']:
        fig = px.bar(top_5(parameter), x=region+'_top', y=region+'_sales_proportion',
                 title='Доли парметра '+parameter+' в регионе '+region)
        fig.show()

Начнем применять наши функции. Сперва **определим самые популярные платформы** для каждого из представленных в данных региона:

In [ ]:
top_5('platform')

In [ ]:
bar_plot('platform')

Заметим различное распределение долей продаж среди 5 самых популярных платформ в каждом регионе, что особенно видно по первой строчке рейтингов. Так, в то время как в na доля продаж лидера среди остальных платформ - четверть, а в eu - почти 40 %, в jp на первую платформу в рейтинге приходится почти половина всех продаж.<br>
В каждый из трех рейтингов попала платформа PS4, PS3 и 3DS. Регион jp единственный содержит в своем рейтинге платформы WiiU и PSV, расположившиется на 5 и 3 местах рейтинга соответственно. Регионы na и eu отличаются друг от друга лишь порядком следования платформ в рейтинге, не считая первой и последней позиции - PS4 и 3DS. <br>
Самые популярные платформы в топ-5 по регионам следующие: na, eu - PS4, jp - 3DS. Самые непопулярные: na, eu - 3DS, jp - WiiU.

Теперь **исследуем самые популярные жанры** для каждого региона.

In [ ]:
top_5('genre')

In [ ]:
bar_plot('genre')

В этом случае видим менее различное распределение долей во всех трех рейтингах. На этот раз отличия между всеми тремя наборами еще меньше: топы na и eu совпадают даже позиционно, отличаясь лишь пятой позицией рейтинга - это Misc и Racing соответственно. Расположение значений в регионе jp является перестановкой расположения в na с заменой Sports на Fighting. Уникальные значения демонстрируют eu и jp - это Racing и Fighting соответственно. Общими для всех трех топов являются жанры Action, Shooter и Role-Playing.<br>
Опишем первые позиции рейтингов: na, eu - Action, jp - Role_Playing. Последние - na - Misc, eu - Racing, jp - Shooter. 

Наконец, **определим наличие влияния рейтинга на продажи** в этих регионах.

In [ ]:
top_5('rating')

In [ ]:
bar_plot('rating')

Видим, что рейтинги для na и eu совпадают даже позиционно, если не обращать внимание на пропущенные значения.  Топ для jp является перестановкой предыдущих двух рейтингов, почти обратной, правда в ней поменяны местами значения E и E10+.<br>
Явным образом прослеживается влияние рейтинга на попадание в топ продаж по региону, причем на это не зависит от региона, так как каждый из трех рейтингов состоит из одних и тех же значений.<br>
Отметим очень похожее распределение долей продаж игр каждого конкретного рейтинга для регионов na и eu.<br> 
Кроме того, бросается в глаза то, что в jp большая часть игр не имеет рейтинга ESRB, предположим, что это связано с тем, что система такой оценки игр была разработана в США и характерна больше для англоязычного мира, в то время как в Японии для оценки игр зачастую используется собственная система рейтинга - CERO.<br> 
Интерсной задачей было бы прогнозирования рейтинга в ячейках с 'unknown' с помощью более твердого исследования влияния всех остальных параметров на этот рейтинг, задача эта, впрочем, относится скорее к области машинного обучения и интеллектуального анализа.

Видим, что в двух случаях из трех (топы по платформе и рейтингу) распределение топовых позиций по долям продаж было различным, при том, что для каждого топа были характерны схожие, если не одинаковые, значения. Обращает на себя большая схожесть каждого топа в регионах na и eu, и их общее отличие от региона jp, что может объясняться более тесными культурными связями между США и Европейским Союзом, и культурной же отдаленностью от них Японии.<br> 
Скорее всего, чтобы более детально обрисовать отличительные особенности каждого регионального рынка потребуется задействовать более точные аналитические инструменты, проводить анализ по большему набору характеристик. 

## 4. Проверка гипотез

Проверим две гипотезы, касающиеся равенства средних двух наборов данных.

Для начала проверим следующую гипотезу - **средние пользовательские рейтинги платформ Xbox One и PC одинаковые** сформулируем ее более строго, выбрав уровень значимости 0.05:

```
H_0: Пользовательские рейтинги (user_score) платформы 'XOne' = пользовательские рейтинги (user_score) платформы 'PC'`
H_a: Пользовательские рейтинги (user_score) платформы 'XOne' ≠ пользовательские рейтинги (user_score) платформы 'PC'`
alpha = 0.05
```

Так как мы проверяем гипотезу о равенстве среднего двух независимых совокупностей данных, применим метод scipy.stats.ttest_ind():

In [ ]:
results = st.ttest_ind(
    games_actual_period.loc[games_actual_period.platform == 'XOne', 'user_score'], 
    games_actual_period.loc[games_actual_period.platform == 'PC', 'user_score'],
    nan_policy='omit'
)
alpha = .05
print(results.pvalue)
if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

Проведенный тест привел к отвержению, среднее рейтингов не одинаково.

Теперь проверим гипотезу, говорящую о том, что **средние пользовательские рейтинги жанров Action и Sports разные**. Опять сформулируем нулевую и альтернативную гипотезы с тем же уровнем значимости:

```
H_0: Пользовательские рейтинги (user_score) жанра 'Action' = пользовательские рейтинги (user_score) жанра 'Sports'`
H_a: Пользовательские рейтинги (user_score) жанра 'Action' ≠ пользовательские рейтинги (user_score) жанра 'Sports'`
alpha = 0.05
```

Здесть мы так же проверяем гипотезу о равенстве среднего двух независимых совокупностей, поэтому снова применим метод scipy.stats.ttest_ind():

In [ ]:
results = st.ttest_ind(
    games_actual_period.loc[games_actual_period.genre == 'Action', 'user_score'], 
    games_actual_period.loc[games_actual_period.genre == 'Sports', 'user_score'],
    nan_policy='omit'
)
alpha = .05
print(results.pvalue)
if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

Этот тест отвергает гипотезу о равенстве средних пользовательских рейтингах, они, судя по всему, различны.

## 5. Общий вывод

В ходе исследования придерживались следующей траектории работы:
1. Обзор и подготовка данных
На этом шаге мы исправляли некоторые недостатки в имеющихся данных. *Приведение столбцов к нижнему регистру* позволило повысить удобство чтения и работы с названиями колонок датафрейма. После этого было произведено *преобразование типов*, в ходе готорого разобрались с логикой устройства датафрейма, оптимизировали использование памяти. На следующем этапе работы занялись *обработкой пропусков*, чтобы максимизировать пользу от анализа данных в нашем наборе, применяя средства визуализации пропусков, срезов данных, анализируя возможные причины отсутствия значений, пришли к решению о том, как действовать с пропусками в каждой отдельной ситуации. Завершая этот шаг работы, *создали новый столбец* в данных, который был необходим на дальнейших аналитических шагах.
2. Исследовательский анализ данных
В течение этого этапа работы были проведены следующие аналитические действия:
    1. С помощью столбчатой диаграммы было оценено количество игр, выходивших в разные годы, сделали вывод, что **для дальнейшего анализа, скорее всего, понадобится лишь часть имеющихся данных**.
    2. Были найдены платформы с наибольшим годовым объемом продаж, снова используя столбчатую диаграмму, проанализировали динамику возникновения на рынке игр для этих платформ, сколько длится период их продаж, какова их "живучесть" на рынке видеоигр. Определили, что **медианная продолжительность нахождения игр для конкретной платформы на рынке составляет 9-10 лет**.
    3. По результата выбрали **актуальный период для исследования - 2013-2016 годы**, который мог бы быть полезен при прогнозировании дальнейших продаж игр по разным параметрам.
    4. С помощью линейного графика проанализировали динамику роста/падения годовых продаж в выбранный период. Выяснилось, что **в наблюдаемый временной интервал происходило стабильное падение продаж игр для всех платформ (но необходимо помнить о некоторых временных исключениях**. По итогу этого этапа выбрали **наиболее перспективные в плане объема продаж платформы - PS4, XOne и 3DS**. 
    5. Построив диаграммы размаха для выбранных платформ, оценили их медианные продажи. **Выше всего средние продажи оказались у платформы XOne**, отметили **скошенность вправо у данных по всем выбранным платформам**. 
    6. Провели исследование влияния оценок игроков и критиков на продажи игр на платформе X360, с помощью построения диаграмм рассеяния и вычисления коэффициента корреляции Пирсона, выяснилось, что **связь с количеством продаж обнаруживается только у оценок критиков**.
    7. Выводы о влиянии отзывов на продажу игр для этой платформы были соотнесены с тем же влиянием на остальные платформы из списка потенциально прибыльных. Определили, что **на продажи влияют оценки критиков, причем коэффициент корреляции для них находится в пределах 0.45, оценки пользователей же имеют на продажи нулевое значение, лишь в случае с платформой 3DS коэффициент корреляции отличился от нуля (на 2 десятых)**. 
    8. Совершив аккуратное сравнение медианных продаж (воспользовались сводными таблицами и диаграммами размахи), провели анализ прибыльности представленных игровых жанров. В реультате, пришли к тому, что **в первой тройке по продажам находятся жанры shooter, sports и platform, а в последней -  puzzle, strategy и adventure**.
3. Формирование портрета потребителя для имеющихся в данных регионов
С помощью написания универсальных функций проанализировали самые популярные платформы и жанры в каждом регионе, оценив доли продаж, им соответствующие. Кроме того, оценили влияние присвоенного игре рейтинга на ее продаваемость. Заметили, что **во всех случаях распределение целевых параметров по долям продаж было весьма разным**, с другой стороны, сам **состав лидеров по продажам во всех случаях был во многом идентичным**. Кроме того, заметили , что **результаты анализа были очень похожи для na и eu, было выдвинуто предположениие о причине отличия от них результатов для jp**, еще, **для региона jp предположили причину, по которой большей части игр не был присвоен рейтинг ESRB**.
4. Проверка гипотез
На последнем шаге проверили гипотезы о равенстве или неравенстве средних пользовательских оценок для конкретных платформ и жанров. Выяснили, что **средние пользовательские рейтинги платформ Xbox One и PC не получается назвать одинаковыми, как и средние пользовательские оценки жанров Action и Sports**, для этого использовали средства библиотеки scipy.

Таким образом, можем заключить, что **в 2017 году можно ожидать дальнейшего падения продаж игр для всех платформ, среди которых судя по всему лучше всего будут продаваться игры на XOne и PS4, обладающие хорошими оценками от критиков. Стоит также рассчитывать на хорошие международные продажи игр в жанрах shooter, sports и platform. Кроме того, не стоит забывать о региональной специфике продаж, особенно в случая с японским рынком видеоигр.** 